<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/Proyecto_DFO_ruido_problemas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt

In [161]:
def L_BFGS(S,Y,g):#Checar las posibles divisiones por cero
    n=len(g)
    m=len(S)
    #supondremos H_0 un multiplo de la identidad
    H_0=(np.dot(S[-1],Y[-1])/(np.max([np.dot(Y[-1],Y[-1]),10**(-6)] )))*np.identity(n)#Dado en clase.
    q=g
    alphas=[]
    for i in range(m):
        alphas.append((1/max(10**(-6),np.dot(S[m-1-i],Y[m-1-i])))*np.dot(S[m-1-i],q))
        q=q-alphas[i]*Y[m-1-i]
    r=np.dot(H_0,q)
    for i in range(m):
        beta=(1/max(10**(-6),np.dot(Y[i],S[i])))*np.dot(Y[i],r)
        r=r+S[i]*(alphas[m-1-i]-beta)#aproximacion de -H*g
    return r
def ECNoise(F):#F es un vector que guarda f(t_i), i=0,...,m. Evaluaciones de m+1 puntos equiespaciados
#Ver metodo en Estimating_Computational_Noise, pagina 8
    m=len(F)-1
    T=np.zeros((m+1,m+1))
    for i in range(m+1):
        T[i,0]=F[i]
    for k in range(m):
        for i in range(m-k):
            T[i,k+1]=T[i+1,k]-T[i,k]
    return T
def var_estimation(T):#Recibe la tabla.
    m=len(T)-1
    sigma_array=[]
    for k in range(1,m):
        sigma_array.append(((np.math.factorial(k)/np.math.factorial(2*k))/(m+1-k))*np.sum(T.T[k]**2))
    #Ahora checamos las 2 condiciones.
    for k in range(1,m-2):
        max_k=np.max(sigma_array[k:k+2])
        min_k=np.min(sigma_array[k:k+2])
        if(max_k<=4*min_k):
            if(np.sign(min_k*max_k)==-1):
                print("es el ", k)
                break
    return np.sqrt(sigma_array[k])#Podria ser que las condiciones no se cumplan, en tal caso regresa la ultima estimacion sigma.
def aprox_f_biprima(e_f,f,x_k,p):#Usando una direccion (aleatoria dada) se aproxima el max ||Hessiana_f(x_k)*p||
    #Ver Estimating Derivatives of Noisy Simulations, pagina 14.
    tao_1=100
    tao_2=0.1
    h_a=e_f**(1/4)
    f_mas=f(x_k+h_a*p)
    f_menos=f(x_k-h_a*p)
    f_0=f(x_k)
    delta_h_a=abs(f_mas+f_menos-2*f_0)
    mu_a=delta_h_a/(np.max([h_a**2,10**(-6)]))
    if(delta_h_a/np.max([e_f,10**(-6)])>=tao_1):
        mu=mu_a
        if(abs(f_mas-f_0)<=tao_2*max(f_mas,f_menos,f_0) and abs(f_menos-f_0)<=tao_2*max(f_mas,f_menos,f_0)):
            return mu_a
    h_b=(e_f/np.max([mu_a,10**(-6)]))**(1/4)
    f_mas=f(x_k+h_b*p)
    f_menos=f(x_k-h_b*p)
    delta_h_b=abs(f_mas+f_menos-2*f_0)
    mu_b=delta_h_b/(np.max([h_b**2,10**(-6)] ))
    if(delta_h_b/np.max([e_f,10**(-6)])>=tao_1):
        mu=mu_b
        if(abs(f_mas-f_0)<=tao_2*max(f_mas,f_menos,f_0) and abs(f_menos-f_0)<=tao_2*max(f_mas,f_menos,f_0)):
            return mu_b
    if(abs(mu_a-mu_b)<=0.5*mu_b):
        return mu_b
    #print("No hay regla de decision")
    return np.max([mu_a,mu_b,10**(-6)])#Por falta de relga de descicion. En el algoritmo no se especifica una opcion.

def rutina_h(f,p,x_0,m=10):#Calcula h a partir de una direccion dada, con m+1 puntos equiespaciados.
    f_0=f(x_0)
    n=len(x_0)
    delta=10**(-3)#Suggested by ECnoise algorithm.
    F_st=[f(x_0+delta*i*p) for i in range(m)]
    T=ECNoise(F_st)
    e_f=var_estimation(T)
    #print("e_f: ",e_f)#Breakpoint
    mu=aprox_f_biprima(e_f=e_f,f=f,x_k=x_0,p=p)
    h=(8**(1/4))*(e_f/np.max([mu,10**(-6)]))**(1/2)
    return e_f,h
def relaxed_Armijo(x_k,f,fprime,d,alpha_0=1,e_f=0.0,c_1=0.1,c_2=0.9,
                   max_iter=10**2):##Intenté usar scipy.line_search(), pero Strong Wolfe son dificiles de satisfacerse
    #Metodo visto en clase.
    iter=0
    alpha=0
    grad_f_k=fprime(x_k)
    f_k=f(x_k)
    beta=np.inf
    alpha_i=alpha_0
    g_dot_d=np.dot(grad_f_k,d)
    Armijo=False
    for i in range(max_iter):
        if(f(x_k+alpha_i*d)>f_k+c_1*alpha_i*g_dot_d+2*e_f):#Aqui esta la condicion relajada, dado e_f estimacion del error estandar.
            beta=alpha_i
            alpha_i=0.5*(alpha+beta)
        else:
            Armijo=True
            if(np.dot(fprime(x_k+alpha_i*d),d)<c_2*g_dot_d):
                alpha=alpha_i
                if(beta==np.inf):
                    alpha_i=2*alpha
                else:
                    alpha_i=0.5*(alpha+beta)
            else:
                break
    if(Armijo==True):
        return alpha_i
    else:
        return Armijo

In [162]:
def recovery(x_k,f,grad_k,e_f,h,l1,l2,d_k,m=10):#Genera un nuevo h
    d_k=d_k/np.max(d_k)
    e_f_b,h_b=rutina_h(f=f,p=d_k,m=m,x_0=x_k)
    n=len(x_k)
    f_stencil=[]
    f_k=f(x_k)
    if(h_b<l1*h or h_b>l2*h):
        return x_k,e_f_b,h_b
    else:
        
        x_h=x_k+h*(d_k)
        f_h=f(x_h)
        if(f_h<=f(x_k)+0.1*h*np.dot(grad_k,d_k)+2*e_f):
            return x_h,e_f,h
        else:
            for i in range(n):
                x_s=x_k
                x_s[i]+=h
                f_stencil.append(f(x_s))
            ii=np.argmin(f_stencil)
            x_s=x_k
            x_s[ii]+=h
            f_s=f(x_s)
            if(f_h<=f_s and f_h<=f_k):
                return x_h,e_f,h
            else:
                if(f_k>f_s and f_h>f_s):
                    return x_s,e_f,h
                else:
                    e_f_n,h_n=rutina_h(f=f,p=np.random.uniform(-1,1,size=n),m=m,x_0=x_k)
                    return x_k,e_f_n,h_n

In [163]:
import time
def find_minimum(f,x_0,tol_g=10**(-4),zeta=0.01,max_iter=10**2,alpha_max_iter=100,m=10,ml=200):
    t1 = time.time()
    n=len(x_0)
    #----------------------
    #Listas
    iter=0
    GG=[]
    Y=[]
    S=[]
    #----------------------
    #Primera iteracion para crear S_k, Y_k------------------------
    e_f,h=rutina_h(f=f,p=np.random.uniform(-1,1,size=len(x_0)),m=m,x_0=x_0)
    #print("h: ",h)
    def fprime(x):
        return scipy.optimize.approx_fprime(xk=x,f=f,epsilon=np.max([h,10**(-6)]))
    g_0=fprime(x_0)#Gradiente
    alpha_k=relaxed_Armijo(f=f,fprime=fprime,x_k=x_0,d=-g_0,e_f=e_f,max_iter=alpha_max_iter)#line_search
    #alpha_k=backtrack(x_k=x_0, f=f, grad=fprime, d_k=-g_0)
    x_1=x_0-alpha_k*g_0#Primera direccion de descenso es -grad(x_0)
    g_1=fprime(x_1)
    GG.append(np.linalg.norm(g_1))
    #----------------------
    #diferencia de gradientes
    #diferencia de x's
    Y.append(g_1-g_0)
    S.append(-alpha_k*g_0)
    #comp=[np.linalg.norm(d_k_r+g_0)]#Lista para compararlos
    #H_k=(np.dot(S[-1],Y[-1])/(np.dot(Y[-1],Y[-1])))*np.identity(n)#Dado en clase.
    #----------------------
    #----------------------
    while(np.linalg.norm(g_1)>tol_g and iter<max_iter):#Criterio de par
        x_0=x_1
        g_0=g_1
        #H_k_m=BFGS(s=S[-1],y=Y[-1],H=H_k)
        #d_k=-np.dot(np.linalg.inv(Rosembrok_Hessian(x_0)),g_0)
        d_k=-L_BFGS(S=S,Y=Y,g=g_1)
        #d_k=d_k/np.max([10**(-6),np.linalg.norm(d_k)])
        #d_k=-np.dot(H_k_m,g_0)
        if(iter>ml and len(Y)>ml-10):#Elimina los viejos s_k,y_k
            del S[0]
            del Y[0]
        alpha_k_try=relaxed_Armijo(f=f,fprime=fprime,x_k=x_0,d=d_k,e_f=e_f,max_iter=alpha_max_iter)
        if(alpha_k_try==False):
            #print("Line search no funcionó")
            x_1,e_f,h=recovery(x_k=x_0,f=f,grad_k=g_1,e_f=e_f,h=h,l1=0.01,l2=100,d_k=d_k,m=12)
            def fprime(x):
                return scipy.optimize.approx_fprime(xk=x,f=f,epsilon=np.max([h,10**(-6)]))
        else:
            x_1=x_0+alpha_k*d_k#
        g_1=fprime(x_1)
        iter+=1
        #H_k=H_k_m
        y_k=g_1-g_0
        s_k=x_1-x_0
        if(np.dot(s_k,y_k)>=zeta*np.linalg.norm(y_k)*np.linalg.norm(s_k)):
            Y.append(y_k)
            S.append(s_k)
        #print(np.linalg.norm(d_k-d_k_l))
        GG.append(np.linalg.norm(g_1))
    print("Summary: ")
    #print("comp:",comp[-1])
    #plt.plot(GG)
    plt.show()
    t2 = time.time()
    tim = t2-t1
    niter = iter
    print("time: {:.02e}\t Niter: {} \t ||g_k||: {:.02e} ".format(
             tim,niter,GG[-1]))
    return x_1

In [164]:
def p_204(X):#x_0=np.array([0.1,0.1] ), x_opt=np.array([0,0] )
    A=np.array([0.13294 ,-0.244378, 0.325895] )
    D=np.array([2.5074 ,-1.36401,1.02282] )
    H=np.array([[-0.564255 ,0.392417], [ -0.404979, 0.927589], [-0.0735084 ,0.535493]] )
    B=np.linalg.inv(np.dot(H.T,H))
    W=A+np.dot(H,X)+0.5*np.dot(X,np.dot(B,X))*D
    return np.dot(W,W)+np.random.uniform(-10**(-2),10**(-2))

In [165]:
def p_205(X):#x_0=np.array([0.1,0.] ), x_opt=np.array([3,0.5] )
    return ((1.5-X[0]*(1-X[1]))**2+(2.25-X[0]*(1-X[1]**2))**2+(2.625-X[0]*(1-X[1]**3))**2)


In [166]:
def p_206(X):#x_0=[-1.2,1], x_opt=[1,1]
    return ((X[1]-X[0]**2)**2+100*(1-X[0])**2)

In [167]:
def p_212(X):#x_0=[2,0], x_opt=[0,0]
    return (((4*(X[0]+X[1]))**2+(4*(X[0]+X[1])+(X[0]-X[1])*((X[0]-2)**2+X[1]**2-1))**2))


In [168]:
def p_214(X):#x_0=[-1.2,1], x_opt=[1,1]
    return ((10*(X[0]-X[1])**2+(X[0]-1)**2)**(1/4))

In [169]:
def p_271(X):#x_0=np.array([0,0,0,0,0,0]),x_opt=np.array([1,1,1,1,1,1])
    suma=0
    for i in range(1,len(X)+1):
        suma+=(16-i)*(X[i-1]-1)**2
    return 10*suma

In [170]:
def p_276(X):#x_0=np.array([-4,-2,-1.333,-1,-0.8,-0.6667]), x_opt=np.zeros(6)
    n=len(X)
    X=np.array(X)
    A=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if(i+j!=1):
                A[i,j]=1/(i+j+1)
    return np.dot(X,np.dot(A,X))

In [171]:
def p_281(X):#x_0=np.array([0,0,0,0,0,0,0,0,0,0]), x_opt=np.ones(10)
    suma=0
    for i in range(1,len(X)+1):
        suma+=(i**3)*(X[i-1]-1)**2
    return suma**(1/3)

In [172]:
def p_282(X):#x_0=[-1.2,0,0,0,0,0,0,0,0,0], x_opt=np.ones(10)
    sum=0
    for i in range(1,len(X)):
        sum+=(10-i)*(X[i-1]**2-X[i])**2
    return (X[0]-1)**2+(X[9]-1)**2+10*sum

In [173]:
def p_283(X):#x_0=np.zeros(10), x_opt=np.ones(10)
    sum=0
    for i in range(1,len(X)+1):
        sum+=(i**3)*(X[i-1]-1)**2
    return sum**3


In [174]:
def p_303(X):#x_0=np.ones(20)*0.1, x_opt=np.zeros(20)
    sum_1=0
    sum_2=0
    sum_3=0
    for i in range(1,len(X)+1):
        sum_1+=X[i-1]**2
        sum_2+=0.5*i*X[i-1]
    return sum_1+sum_2**2+sum_2**4

In [175]:
def p_ros(X):
    return (np.sum(100*(X[1:]-X[:-1]**2)**2+(1-X[:-1])**2))

In [176]:
def additive_noise(f,x,eps=10**(-2),label='stochastic'):
    if(label=='stochastic'):
        u=np.random.uniform(-eps,eps)
    return(f(x)+u)
def multiplicative_noise(f,x,eps=10**(-2),label='stochastic'):
    if(label=='stochastic'):
        u=np.random.uniform(-eps,eps)
    return(f(x)*(1+u))

In [177]:
def f_noise(f,x,eps,noise):
        if(noise=='additive'):
            return additive_noise(f=f,eps=eps,x=x)
        if(noise=='multiplicative'):
            return multiplicative_noise(f=f,eps=eps,x=x)

In [178]:
x_0=np.array([-1.2,0.5])
def f_1(x):
    return f_noise(f=p_206,eps=10**(-2),x=x,noise='additive')
x_opt=find_minimum(f=f_1,x_0=np.array(x_0),tol_g=10**(-3),max_iter=10**3,alpha_max_iter=10)

Summary: 
time: 2.91e+00	 Niter: 1000 	 ||g_k||: 1.23e+02 


In [179]:
#Funciones de prueba con ruido.
funciones=[p_204,p_205,p_206,p_212,p_271,p_276,p_281,p_282,p_283,p_303,p_ros]
x_initial=[np.array([0.1,0.1] ),np.array([0.1,0.] ),[-1.2,1],[2,0],np.array([0,0,0,0,0,0]),np.array([-4,-2,-1.333,-1,-0.8,-0.6667]),np.array([0,0,0,0,0,0,0,0,0,0]),[-1.2,0,0,0,0,0,0,0,0,0],np.zeros(10), np.ones(20)*0.1, np.zeros(50) ]
x_optims=np.array([[0,0],[3,0.5],[1,1],[0,0] ,np.array([1,1,1,1,1,1]),np.zeros(6),np.ones(10),np.ones(10),np.ones(10),np.zeros(20),np.ones(50)])
noise_type=['additive']
for i in range(len(funciones)):
    f=funciones[i]
    print("Function: ", str(f))
    x_0=x_initial[i]
    n=len(x_0)
    for noise in noise_type:
        print(noise+' Case\n')
        def f_1(x):
            return f_noise(f=f,eps=10**(-2),x=x,noise=noise)
        x_opt=find_minimum(f=f_1,x_0=np.array(x_0),tol_g=10**(-3),max_iter=10**3,alpha_max_iter=10)
        print("||x_opt-x^*||:",np.linalg.norm(x_opt-x_optims[i]))
        print("--------------------------")
    print("---------------------------------")


Function:  <function p_204 at 0x7f832ea64cb0>
additive Case



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


Summary: 
time: 6.60e+00	 Niter: 1000 	 ||g_k||: 2.01e+00 
||x_opt-x^*||: 0.11053039503466018
--------------------------
---------------------------------
Function:  <function p_205 at 0x7f832eab64d0>
additive Case

Summary: 
time: 2.72e+00	 Niter: 1000 	 ||g_k||: 1.69e+02 
||x_opt-x^*||: 4.3307985975015715
--------------------------
---------------------------------
Function:  <function p_206 at 0x7f832eab6560>
additive Case

Summary: 
time: 2.63e+00	 Niter: 1000 	 ||g_k||: 1.56e+02 
||x_opt-x^*||: 1.6772064747977113
--------------------------
---------------------------------
Function:  <function p_212 at 0x7f832eab6f80>
additive Case

Summary: 
time: 2.57e+00	 Niter: 1000 	 ||g_k||: 1.67e+01 
||x_opt-x^*||: 1.1696470100587693
--------------------------
---------------------------------
Function:  <function p_271 at 0x7f832d17e710>
additive Case

Summary: 
time: 2.90e+00	 Niter: 1000 	 ||g_k||: 9.26e+01 
||x_opt-x^*||: 0.29802736124578333
--------------------------
------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


Summary: 
time: 3.32e+00	 Niter: 1000 	 ||g_k||: 7.72e+10 
||x_opt-x^*||: 3.1622645084936942
--------------------------
---------------------------------
Function:  <function p_303 at 0x7f832d16d680>
additive Case

Summary: 
time: 4.26e+00	 Niter: 1000 	 ||g_k||: 1.39e+05 
||x_opt-x^*||: 0.4610104783085393
--------------------------
---------------------------------
Function:  <function p_ros at 0x7f832d16dcb0>
additive Case

Summary: 
time: 6.33e+00	 Niter: 1000 	 ||g_k||: 3.98e+02 
||x_opt-x^*||: 7.013351390978638
--------------------------
---------------------------------
